# Step 7: Performance interpretability: 
## Decompose your preferred performance metric using Permutation Importance and XPER. Compare the two sets of results.

In [8]:
import pandas as pd
import xgboost as xgb
from sklearn.model_selection import train_test_split
from sklearn.dummy import DummyClassifier
from sklearn.metrics import accuracy_score, balanced_accuracy_score, roc_auc_score, f1_score
from sklearn.model_selection import GridSearchCV
import seaborn as sns
import numpy as np
import matplotlib.pyplot as plt
import pickle

from mlxtend.evaluate import feature_importance_permutation

In [17]:
raw_data = pd.read_csv("../data/data_clean.csv")
data = raw_data[raw_data['birth date'] < 2020].copy()

data['age'] = data['parole board interview date'] - data['birth date']
data['jail duration'] = data['parole board interview date'] - data['year of entry']

others_parole_type =  ['PIE', 'SP CONSDR', 'ECPDO', 'MEDICAL','RESCISSION', 'DEPORT']
data['parole board interview type'] = data['parole board interview type'].replace(others_parole_type, 'OTHERS').replace('SUPP MERIT', 'MERIT TIME').replace('PV REAPP', 'REAPPEAR')

data = data.dropna(axis=0, subset=['crime 1 - class', 'parole eligibility date'])

df_one_hot = pd.get_dummies(data, columns=[
    "sex", "race / ethnicity"], drop_first=True)

df_one_hot = pd.get_dummies(df_one_hot, columns=[
    "crime 1 - class", "crime 2 - class",
    "crime 3 - class", "crime 4 - class",
    "parole board interview type"])

df_one_hot.drop(columns=['release date','birth date', 'year of entry'],inplace=True)

X_train, X_test, y_train, y_test = train_test_split(df_one_hot.drop('y',axis=1), df_one_hot['y'], 
                                                    stratify=df_one_hot['y'], test_size=0.3, random_state=42)

In [18]:
model_xgb = pickle.load(open('../model.pkl', 'rb'))
model_xgb

XGBClassifier(base_score=None, booster=None, callbacks=None,
              colsample_bylevel=None, colsample_bynode=None,
              colsample_bytree=None, device=None, early_stopping_rounds=None,
              enable_categorical=False, eval_metric=None, feature_types=None,
              gamma=None, grow_policy=None, importance_type=None,
              interaction_constraints=None, learning_rate=0.1, max_bin=None,
              max_cat_threshold=None, max_cat_to_onehot=None,
              max_delta_step=None, max_depth=3, max_leaves=None,
              min_child_weight=None, missing=nan, monotone_constraints=None,
              multi_strategy=None, n_estimators=500, n_jobs=None,
              num_parallel_tree=None, random_state=42, ...)

In [19]:
X_test

,parole board interview date,parole eligibility date,min_sentence,max_sentence,count of crimes,age,jail duration,sex_MALE,race / ethnicity_ASIAN/PACIFIC,race / ethnicity_BLACK,...,crime 3 - class_E,crime 4 - class_A,crime 4 - class_B,crime 4 - class_C,crime 4 - class_D,crime 4 - class_E,parole board interview type_INITIAL,parole board interview type_MERIT TIME,parole board interview type_OTHERS,parole board interview type_REAPPEAR
14045,2014,2014.0,6.0,3.0,1,36.0,2,1,0,0,...,0,0,0,0,0,0,1,0,0,0
21177,2014,2015.0,2.0,4.0,1,34.0,1,1,0,1,...,0,0,0,0,0,0,1,0,0,0
21333,2014,2017.0,5.0,6.0,1,34.0,3,1,0,1,...,0,0,0,0,0,0,0,0,1,0
20664,2014,2015.0,4.0,3.0,2,45.0,3,1,0,0,...,0,0,0,0,0,0,1,0,0,0
15941,2014,2008.0,6.0,12.0,1,39.0,11,1,0,1,...,0,0,0,0,0,0,0,0,0,1
...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...
8746,2013,2011.0,20.0,99.0,1,46.0,21,0,0,1,...,0,0,0,0,0,0,0,0,0,1
26977,2015,2015.0,6.0,3.0,2,43.0,1,1,0,0,...,0,0,0,0,0,0,1,0,0,0
24655,2015,2015.0,2.0,6.0,1,41.0,2,1,0,1,...,0,0,0,0,0,0,1,0,0,0
28375,2015,2016.0,6.0,7.0,2,48.0,2,1,0,1,...,0,0,0,0,0,0,1,0,0,0


In [10]:
imp_vals, _ = feature_importance_permutation(
    predict_method=model_xgb.predict, 
    X=X_test,
    y=y_test,
    metric='accuracy',
    num_rounds=1,
    seed=1)

imp_vals

InvalidIndexError: (slice(None, None, None), 0)